IIT GANDHI NAGAR TASK :- to analyze a thermal video

In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture('input_video.mp4')

n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # frames caluclated 

fps = cap.get(cv2.CAP_PROP_FPS) #fps of the video 

min_frequency = 0 #minimum frequency
max_frequency = 1 #maximum frequency
# limit for fourier domain

Adding input file to the open.cv and Calculation of frame count 

In [3]:
for _ in range(n_frames): #reading frame by frame
    ret, frame = cap.read() #each fra,e is initialized
    
    if not ret:    #if frame is not read then break the loop
        break
    
    #converts to grayscale frame 
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    #transforms fourier and shifts the 0 frequency to center of spectrum 
    f_transform = np.fft.fft2(gray_frame)
    f_transform_shifted = np.fft.fftshift(f_transform)
    
    #
    rows, cols = gray_frame.shape
    frequency_x = np.fft.fftfreq(cols, d=1/fps)
    frequency_y = np.fft.fftfreq(rows, d=1/fps)
    
    frequency_x_hz = np.fft.fftshift(frequency_x) * fps  
    frequency_y_hz = np.fft.fftshift(frequency_y) * fps  
    
    mask_x = np.logical_and(np.abs(frequency_x_hz) >= min_frequency, np.abs(frequency_x_hz) <= max_frequency)
    mask_y = np.logical_and(np.abs(frequency_y_hz) >= min_frequency, np.abs(frequency_y_hz) <= max_frequency)
    
    f_transform_shifted[~mask_y, :] = 0
    f_transform_shifted[:, ~mask_x] = 0
    
    #normalization of fourier domain 
    filtered_frame = np.fft.ifft2(np.fft.ifftshift(f_transform_shifted)).real
    
    filtered_frame = cv2.normalize(filtered_frame, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U) 
    
    cv2.imshow('Filtered Frame', cv2.cvtColor(filtered_frame,cv2.COLOR_GRAY2BGR)) #display of ouput 
    
    if cv2.waitKey(10) & 0xFF == ord('q'): #if frames finishes or q is pressed the loop will break 
        break                         # vidoe will no longer be played 

cap.release()
cv2.destroyAllWindows() #destroying the rest windows